## MimiBRICK Run Hindcasts and Projections Examples

This notebook contains examples of the hindcast and projection functionalities in the MimiBRICK.jl package.

### Step 1. Environment Setup

For this example, this notebook will run using the environment defined by the `Manifest.toml` and `Project.toml` files within this `examples` folder.  If you wish to follow along and type these out on your local machine, you will need to set up a matching Julia environment. Since MimiBRICK is the only required package, this is most easily done with the following code in your REPL:

`using Pkg`

and 

`Pkg.add(MimiBRICK)`


or equivalently use the `]` key to enter the Pkg REPL, then `add MimiBRICK` followed by backspace to exit the Pkg REPL. 

The following activates the environment within this notebook.

In [ ]:
# Activate the examples environment 
using Pkg # here in case you didn't need to run the above command to instantiate
Pkg.activate(".")
Pkg.instantiate()
Pkg.status() # Check which packages are loaded

Now we use the following to load `MimiBRICK`

In [ ]:
using MimiBRICK

### Step 2. Directories and Data

First we define and create the directory that will hold the results. In this case we will create a folder `hindcast_projections_results` in the current `examples` directory.

In [ ]:
my_output_dir = joinpath(@__DIR__, "hindcast_projections_results")

# if the path already exists, clean it out
isdir(my_output_dir) ? rm(my_output_dir, recursive = true) : nothing

# recreate the folder
mkpath(my_output_dir);

Next we can download the necessary files from our Zenodo repository into this folder. This example uses the parameter samples for the BRICK standalone simulations. Parameter sets for the other model configurations (DOECLIM-BRICK and SNEASY-BRICK) can be found in the Zenodo repository here: https://zenodo.org/record/6626335.

Alternatively, you can use your own previous calibration results (say, some that you created from the `Calibration_Example.ipynb` notebook).

In [ ]:
# get the parameters themselves
filename = "parameters_subsample_brick.csv"
url = "https://zenodo.org/record/6626335/files/"*filename
download(url, joinpath(my_output_dir,filename))

# and the log-posterior scores
filename = "log_post_subsample_brick.csv"
url = "https://zenodo.org/record/6626335/files/"*filename
download(url, joinpath(my_output_dir,filename))

### Step 3. Run Hindcast

First we run the hindcast period 1850-2017 using the `MimiBRICK.run_hindcast` function, for `model_config = brick`. For the hindcast, no RCP scenario needs to be specified, because all of them follow historical radiative forcing/emissions trends up to 2005. We will not set the `start_year` and `end_year` arguments explicitly, as we run default settings here.

The standard set of parameters that are being used for the hindcast and projection simulations are the sub-sample of 10,000 from the MCMC calibration described above (`parameters_subsample_(model_config).csv`).

This script will add model configuration-specific directory that was constructed above (or came with the model codes). It will create a sub-directory called `hindcast_csv` which will be populated with CSV files that include the simulated hindcasts of the model output fields. Each of these names is appended with `model_config` (`brick`, `doeclimbrick`, or `sneasybrick`) and contains one hindcast simulation for each of the sets of parameters in the sub-sample for analysis. Rows correspond to different years (1850-2017 be default) and columns each correspond to different ensemble members.

* `hindcast_antarctic_(model_config).csv` - contribution to global mean sea-level change from the Antarctic ice sheet (meters)
* `hindcast_greenland_(model_config).csv` - contribution to global mean sea-level change from the Greenland ice sheet (meters)
* `hindcast_glaciers_(model_config).csv` - contribution to global mean sea-level change from glaciers and ice caps (meters)
* `hindcast_landwater_storage_sl_(model_config).csv` - contribution to sea-level change from land water storage (meters)
* `hindcast_gmsl_(model_config).csv` - total global mean sea-level change (meters)
* `hindcast_ocean_heat_(model_config).csv` - (DOECLIM- or SNEASY-BRICK only)
* `hindcast_temperature_(model_config).csv` - (DOECLIM- or SNEASY-BRICK only)
* `hindcast_oceanco2_(model_config).csv` - (SNEASY-BRICK only)
* `hindcast_co2_(model_config).csv` - (SNEASY-BRICK only)
* `hindcast_MAP_(model_config).csv` - all of the hindcast time series for the maximum _a posteriori_ set of parameters

The `MimiBRICK.run_hincast` function signature is as follows:

        run_hindcast(; output_dir::String,
                        model_config::String = "brick",
                        start_year::Int = 1850,
                        end_year = 2017,
                    )

_Arguments:_

- _outdir - paths for results files - subsample of model parameters, and associated log-posterior scores, and printed results of this function_
- _model_config (default = "brick") - model configuration with possible options: (1) "brick", (2) "doeclimbrick", (3) "sneasybrick"_
- _start_year (default = 1850) - start year for calibration_
- _end_year (default = 2017) - end year for calibration_

In [ ]:
MimiBRICK.run_hindcast(output_dir=my_output_dir, model_config="brick");

### Step 4. Run Projections

Next we run the projections for the period 1850-2300 (but can be modified to any period between 1765 and 2300) by using the `MimiBRICK.run_projections` function, using `model_config=brick`, `doeclimbrick` or `sneasybrick` and `rcp_scenario="RCP26"`, `"RCP45"`, `"RCP60"`, or `"RCP85"`. Note that the RCP scenario forcing files are all the same until 2005, and the provided stand-alone BRICK temperature and ocean heat forcing files cover the period 1850-2300.

This script will add to the model configuration-specific directory that was constructed above (or came with the model codes). It will create a sub-directory called `projections_csv`, and a sub-directory within there that is specific to each RCP scenario used will be created. The projections files are analogous to the hindcast files that are generated, and will populate the `projections_csv/[RCP scenario]` directory.

The `MimiBRICK.run_projections` function signature is as follows:

        run_projections(; output_dir::String,
                            model_config::String = "brick",
                            rcp_scenario::String = "RCP85",
                            start_year::Int = 1850,
                            end_year = 2300,
                        )
_Arguments:_

- _outdir - paths for results files - subsample of model parameters, and associated log-posterior scores, and printed results of this function_
- _model_config (default = "brick") - model configuration with possible options: (1) "brick", (2) "doeclimbrick", (3) "sneasybrick"_
- _rcp_scenario (default = "RCP85) - RCP scenario with possible options: (1) RCP26, (2) RCP45, (3) RCP60, (4) RCP85_
- _start_year (default = 1850) - start year for calibration_
- _end_year (default = 2300) - end year for calibration_

In [ ]:
MimiBRICK.run_projections(output_dir=my_output_dir, model_config="brick");

#### 